<a href="https://colab.research.google.com/github/honeysilitonga/tugas2/blob/main/Honey_final_project_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import library yang dibutuhkan

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, RobustScaler, StandardScaler
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_curve, precision_recall_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import spearmanr
from sklearn.feature_selection import mutual_info_classif
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

akses ke google drive

In [2]:
from google.colab import drive
drive.mount('/drive/', force_remount=True)

Mounted at /drive/


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ecommerce_traintest.csv')

In [5]:
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ecommerce_pengujian.csv')

(1) Download dataset pada link :

(2) Upload di drive masing-masing

(3) gunakan data ecommerce_traintest.csv, drop kolom yang tidak dibutuhkan.

(4) Define target/output = "label", feature/input kolom lainnya

(5) Split data menjadi training set dan test set (test_size = 0.2) dengan stratify pada kolom **label**

(6) print shape pada X_train, X_test, y_train, dan y_test

In [6]:
# Menampilkan nama kolom untuk dataset pelatihan
kolom_train = df_train.columns
print("Nama Kolom pada Dataset Pelatihan:")
print(kolom_train)

Nama Kolom pada Dataset Pelatihan:
Index(['customer_id', 'customer_city', 'order_status',
       'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'order_item_id', 'shipping_limit_date',
       'price', 'freight_value', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value', 'product_category_name',
       'product_name_lenght', 'product_description_lenght',
       'product_photos_qty', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm', 'label'],
      dtype='object')


In [7]:
# Menampilkan nama kolom untuk dataset pengujian
kolom_test = df_test.columns
print("\nNama Kolom pada Dataset Pengujian:")
print(kolom_test)


Nama Kolom pada Dataset Pengujian:
Index(['customer_id', 'customer_city', 'order_status',
       'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'order_item_id', 'shipping_limit_date',
       'price', 'freight_value', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value', 'product_category_name',
       'product_name_lenght', 'product_description_lenght',
       'product_photos_qty', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm'],
      dtype='object')


# Imbalance Data Handling

In [8]:
# Menampilkan jumlah kolom sebelum melakukan drop pada dataset pelatihan
jumlah_kolom_sebelum_drop_train = df_train.shape[1]
print(f"Jumlah Kolom sebelum Drop pada Dataset Pelatihan: {jumlah_kolom_sebelum_drop_train}")

# Menampilkan jumlah kolom sebelum melakukan drop pada dataset pengujian
jumlah_kolom_sebelum_drop_test = df_test.shape[1]
print(f"Jumlah Kolom sebelum Drop pada Dataset Pengujian: {jumlah_kolom_sebelum_drop_test}")

Jumlah Kolom sebelum Drop pada Dataset Pelatihan: 25
Jumlah Kolom sebelum Drop pada Dataset Pengujian: 24


In [9]:
# Menemukan kolom-kolom yang berbeda antara kedua dataset
kolom_beda_train_test = set(df_train.columns) - set(df_test.columns)
kolom_beda_test_train = set(df_test.columns) - set(df_train.columns)

print("Kolom yang Berbeda dari Dataset Pelatihan ke Dataset Pengujian:")
print(kolom_beda_train_test)

print("\nKolom yang Berbeda dari Dataset Pengujian ke Dataset Pelatihan:")
print(kolom_beda_test_train)

Kolom yang Berbeda dari Dataset Pelatihan ke Dataset Pengujian:
{'label'}

Kolom yang Berbeda dari Dataset Pengujian ke Dataset Pelatihan:
set()


In [10]:
# Menyimpan variabel target
y_train = df_train['label']

In [13]:
# Kolom-kolom yang akan di-drop dari kedua dataset
kolom_tidak_dibutuhkan = ['customer_id',
    'order_status', 'order_purchase_timestamp', 'order_approved_at',
    'order_delivered_carrier_date', 'order_delivered_customer_date',
    'order_estimated_delivery_date', 'order_item_id', 'shipping_limit_date',
    'price', 'freight_value',
    'payment_sequential', 'payment_type', 'payment_installments', 'payment_value']

In [14]:
# Mendrop kolom-kolom yang tidak dibutuhkan dari kedua dataset
X_train = df_train.drop(kolom_tidak_dibutuhkan + ['label'], axis=1)
X_test = df_test.drop(kolom_tidak_dibutuhkan, axis=1)

In [15]:
# Bagi data df_train menjadi set pelatihan dan test dengan stratify
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

In [16]:
# Menampilkan shape dari X_train, X_test, y_train, dan y_test
print("Shape X_train:", X_train.shape)
print("Shape X_test:", X_test.shape)
print("Shape y_train:", y_train.shape)
print("Shape y_test:", y_test.shape)

Shape X_train: (3200, 9)
Shape X_test: (800, 9)
Shape y_train: (3200,)
Shape y_test: (800,)


In [17]:
# Menampilkan beberapa baris pertama dari X_train dan X_test
print("Head X_train:\n", X_train.head())
print("\nHead X_test:\n", X_test.head())

Head X_train:
        customer_city product_category_name  product_name_lenght  \
1045           lages    eletrodomesticos_2                 33.0   
3557  rio de janeiro           eletronicos                 44.0   
3418         valenca             telefonia                 55.0   
24    rio de janeiro      moveis_decoracao                 60.0   
3765          maceio      moveis_decoracao                 45.0   

      product_description_lenght  product_photos_qty  product_weight_g  \
1045                       111.0                 1.0              7350   
3557                       293.0                 3.0             16850   
3418                      2139.0                 2.0               421   
24                         473.0                 1.0               600   
3765                       660.0                 1.0              3450   

      product_length_cm  product_height_cm  product_width_cm  
1045                 44                 14                30  
3557       

(1) Cek nilai sebaran output/target

(2) lakukan imbalance data handling (oversampling atau undersampling)

(3) fit_resample dan dapatkan dataset baru yang dengan nilai output yang balance

In [18]:
# Cek sebaran nilai output/target pada dataset pelatihan
print("Sebaran nilai output pada dataset pelatihan:")
print(y_train.value_counts())

# Cek sebaran nilai output/target pada dataset pengujian
print("\nSebaran nilai output pada dataset pengujian:")
print(y_test.value_counts())

Sebaran nilai output pada dataset pelatihan:
1    1858
0    1342
Name: label, dtype: int64

Sebaran nilai output pada dataset pengujian:
1    465
0    335
Name: label, dtype: int64


In [19]:
print("Dimensi X_train sebelum oversampling:", X_train.shape)
print("Dimensi y_train sebelum oversampling:", y_train.shape)

Dimensi X_train sebelum oversampling: (3200, 9)
Dimensi y_train sebelum oversampling: (3200,)


In [20]:
# Menggabungkan X_train dan y_train menjadi satu DataFrame
df_train_combined = pd.concat([X_train, y_train], axis=1)

# Pisahkan data antara label 0 dan 1
df_label_0 = df_train_combined[df_train_combined['label'] == 0]
df_label_1 = df_train_combined[df_train_combined['label'] == 1]

# Tentukan target jumlah sampel yang diinginkan (misalnya, menyamakan jumlah sampel pada label 0 dan label 1)
target_jumlah_sampel = df_train_combined['label'].value_counts().min()

# Oversampling pada label 0 dan label 1
df_label_0_oversampled = df_label_0.sample(target_jumlah_sampel, replace=True, random_state=42)
df_label_1_oversampled = df_label_1.sample(target_jumlah_sampel, replace=True, random_state=42)

# Gabungkan kembali dataset oversampled
df_train_oversampled = pd.concat([df_label_0_oversampled, df_label_1_oversampled], axis=0)

# Pisahkan kembali fitur dan label
X_train_oversampled = df_train_oversampled.drop('label', axis=1)
y_train_oversampled = df_train_oversampled['label']

# Imputation

(1) Cek missing value pada x_train

(2) lakukan imputation dengan menggunakan KNNImputer (jika perlu)

In [21]:
print(X_train_oversampled.isnull().sum())

customer_city                  0
product_category_name         33
product_name_lenght           33
product_description_lenght    33
product_photos_qty            33
product_weight_g               0
product_length_cm              0
product_height_cm              0
product_width_cm               0
dtype: int64


In [22]:
# Mengisi nilai NaN dengan mean pada setiap kolom menggunakan nilai mean dari X_train_oversampled
X_test = X_test.fillna(X_train_oversampled.mean())

# Transformation

(1) Lakukan transformation :
- data number pake scalling
- data categorical pake OHE

(2) Pada kolom kategorikal X_train lakukan OHE pada parameter handle_unknown beri argumen 'ignore' agar encoder robust terhadap adanya kategori di testing set yang tidak ada di training set.

In [23]:
# Mengetahui kolom pada df_train setelah drop
kolom_df_train_setelah_drop = df_train.columns
print("Kolom pada df_train setelah drop:", kolom_df_train_setelah_drop)

Kolom pada df_train setelah drop: Index(['customer_id', 'customer_city', 'order_status',
       'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'order_item_id', 'shipping_limit_date',
       'price', 'freight_value', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value', 'product_category_name',
       'product_name_lenght', 'product_description_lenght',
       'product_photos_qty', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm', 'label'],
      dtype='object')


In [24]:
# Mengetahui kolom pada X_train setelah drop
kolom_X_train_setelah_drop = X_train.columns
print("Kolom pada X_train setelah drop:", kolom_X_train_setelah_drop)

Kolom pada X_train setelah drop: Index(['customer_city', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm'],
      dtype='object')


In [28]:
# Menampilkan jenis data dari masing-masing kolom pada DataFrame X_train
print(X_train[X_train.columns].dtypes)

customer_city                  object
product_category_name          object
product_name_lenght           float64
product_description_lenght    float64
product_photos_qty            float64
product_weight_g                int64
product_length_cm               int64
product_height_cm               int64
product_width_cm                int64
dtype: object


In [31]:
# Inisialisasi OneHotEncoder
encoder = OneHotEncoder(drop='first', sparse=False, handle_unknown='ignore')

# Fit dan transform fitur kategorikal pada DataFrame X_train
X_train_encoded = encoder.fit_transform(X_train[['customer_city', 'product_category_name']])

# Transform fitur kategorikal pada DataFrame X_test
X_test_encoded = encoder.transform(X_test[['customer_city', 'product_category_name']])

In [32]:
from sklearn.preprocessing import MinMaxScaler

# Inisialisasi MinMaxScaler
scaler = MinMaxScaler()

# Normalisasi fitur numerik pada DataFrame X_train
X_train_numeric_scaled = pd.DataFrame(scaler.fit_transform(X_train[['product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']]), columns=['product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm'])

# Normalisasi fitur numerik pada DataFrame X_test menggunakan skalar dari X_train
X_test_numeric_scaled = pd.DataFrame(scaler.transform(X_test[['product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']]), columns=['product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm'])

In [33]:
# Gabungkan fitur-fitur yang telah di-transformasi untuk X_train
X_train_combined = pd.concat([X_train_numeric_scaled, pd.DataFrame(X_train_encoded, columns=encoder.get_feature_names_out(['customer_city', 'product_category_name']))], axis=1)

# Gabungkan fitur-fitur yang telah di-transformasi untuk X_test
X_test_combined = pd.concat([X_test_numeric_scaled, pd.DataFrame(X_test_encoded, columns=encoder.get_feature_names_out(['customer_city', 'product_category_name']))], axis=1)

# Feature Selection

(1) Lakukan feature selection menggunakan multicolinearity reduction dan mutual info.

In [35]:
# Mengganti nilai kosong dengan nilai rata-rata pada df_train
X_train = X_train.fillna(X_train.mean())

# Mengganti nilai kosong dengan nilai rata-rata pada df_test
X_test = X_test.fillna(X_test.mean())

# Mengganti nilai tak terhingga dengan nilai rata-rata pada df_train
X_train = X_train.replace([np.inf, -np.inf], np.nan)
X_train = X_train.fillna(X_train.mean())

# Mengganti nilai tak terhingga dengan nilai rata-rata pada df_test
X_test = X_test.replace([np.inf, -np.inf], np.nan)
X_test = X_test.fillna(X_test.mean())

In [46]:
print(X_train.columns)

Index(['customer_city', 'product_category_name', 'product_description_lenght',
       'product_photos_qty', 'product_weight_g', 'product_height_cm'],
      dtype='object')


In [39]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import mutual_info_classif

# Ambang batas VIF dan Mutual Information
ambang_batas_vif = 5
ambang_batas_mi = 0.1

In [51]:
# 1. Multicolinearity Reduction (VIF) - df_train
numerical_features_train_vif = ['product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_height_cm']
vif_train = pd.DataFrame()
vif_train["Feature"] = X_train[numerical_features_train_vif].columns
vif_train["VIF"] = [variance_inflation_factor(X_train[numerical_features_train_vif].values, i) for i in range(X_train[numerical_features_train_vif].shape[1])]

In [52]:
# Pilih fitur-fitur dengan VIF yang melebihi ambang batas
high_vif_features_train = vif_train[vif_train["VIF"] > ambang_batas_vif].Feature.tolist()
X_train = X_train.drop(columns=high_vif_features_train)
X_test = X_test.drop(columns=high_vif_features_train)

In [56]:
# 2. Mutual Information - df_train
# Pastikan hanya menggunakan fitur numerik untuk perhitungan Mutual Information
numerical_features_train = ['product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_height_cm']
mutual_info_values_train = mutual_info_classif(X_train[numerical_features_train], y_train)

In [58]:
# Pilih fitur-fitur dengan mutual information yang kurang dari ambang batas
low_mi_features_train = [feature for feature, mi in zip(numerical_features_train, mutual_info_values_train) if mi < ambang_batas_mi]
X_train = X_train.drop(columns=low_mi_features_train)
X_test = X_test.drop(columns=low_mi_features_train)

# Modelling

(1) Buatlah model Logistic Regression, Decision Tree, Random Forest, XGBoost, dan deep learning

(2) Lakukan fitting untuk training dataset

(3) Buat sebuah function classif yang bisa mencetak nilai metrics evaluation

(4) Lakukan evaluasi pada masing-masing model

In [60]:
from xgboost import XGBClassifier

In [68]:
X_train_encoded = pd.get_dummies(X_train, columns=['customer_city', 'product_category_name'], drop_first=True)
X_test_encoded = pd.get_dummies(X_test, columns=['customer_city', 'product_category_name'], drop_first=True)

In [76]:


# Inisialisasi model
logistic_model = LogisticRegression(random_state=42)
decision_tree_model = DecisionTreeClassifier(random_state=42)
random_forest_model = RandomForestClassifier(random_state=42)
xgboost_model = XGBClassifier(random_state=42)

In [77]:
# Fitting model pada training dataset
logistic_model.fit(X_train_encoded, y_train)
decision_tree_model.fit(X_train_encoded, y_train)
random_forest_model.fit(X_train_encoded, y_train)
xgboost_model.fit(X_train_encoded, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [79]:


def evaluate_model(model, X, y):
    # Prediksi pada dataset
    y_pred = model.predict(X)

    # Metrik evaluasi
    accuracy = accuracy_score(y, y_pred)
    precision = precision_score(y, y_pred)
    recall = recall_score(y, y_pred)
    f1 = f1_score(y, y_pred)
    confusion = confusion_matrix(y, y_pred)

    # Cetak hasil evaluasi
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("Confusion Matrix:")
    print(confusion)

# Evaluasi Logistic Regression
print("Logistic Regression:")
evaluate_model(logistic_model, X_train_encoded, y_train)
print("\n")

# Evaluasi Decision Tree
print("Decision Tree:")
evaluate_model(decision_tree_model, X_train_encoded, y_train)
print("\n")

# Evaluasi Random Forest
print("Random Forest:")
evaluate_model(random_forest_model, X_train_encoded, y_train)
print("\n")

# Evaluasi XGBoost
print("XGBoost:")
evaluate_model(xgboost_model, X_train_encoded, y_train)

Logistic Regression:
Accuracy: 0.6787
Precision: 0.6659
Recall: 0.8967
F1 Score: 0.7642
Confusion Matrix:
[[ 506  836]
 [ 192 1666]]


Decision Tree:
Accuracy: 0.8381
Precision: 0.8556
Recall: 0.8676
F1 Score: 0.8616
Confusion Matrix:
[[1070  272]
 [ 246 1612]]


Random Forest:
Accuracy: 0.8381
Precision: 0.8234
Recall: 0.9182
F1 Score: 0.8682
Confusion Matrix:
[[ 976  366]
 [ 152 1706]]


XGBoost:
Accuracy: 0.6362
Precision: 0.6295
Recall: 0.9080
F1 Score: 0.7435
Confusion Matrix:
[[ 349  993]
 [ 171 1687]]


# Cross-Validation

(1) Lakukan cross-validation pada masing-masing model. (Terserah mau pake GSCV atau secara sequential)

(2) Gunakan parameter yang diperoleh dari proses validasi untuk membuat model optimasi.

(3) Bandingkan metrics evaluasi dengan model yang sebelumnya. Model mana yang mendapatkan performa terbaik? Berikan analisis kalian

In [89]:
print("Number of features in X_train_encoded:", X_train_encoded.shape[1])
print("Number of features in X_test_encoded:", X_test_encoded.shape[1])

Number of features in X_train_encoded: 661
Number of features in X_test_encoded: 662


In [90]:
# Convert NumPy arrays to Pandas DataFrame
X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=encoder.get_feature_names_out(['customer_city', 'product_category_name']))
X_test_encoded_df = pd.DataFrame(X_test_encoded, columns=encoder.get_feature_names_out(['customer_city', 'product_category_name']))

# Identify missing features
missing_features = set(X_train_encoded_df.columns) - set(X_test_encoded_df.columns)

# Handle missing features in the test set
X_test_encoded_df = X_test_encoded_df.assign(**{feature: 0 for feature in missing_features})

In [96]:
# Identifikasi indeks fitur yang hilang pada X_train_encoded
missing_features_train = np.setdiff1d(np.arange(X_test_encoded.shape[1]), np.arange(X_train_encoded.shape[1]))

# Identifikasi indeks fitur yang hilang pada X_test_encoded
missing_features_test = np.setdiff1d(np.arange(X_train_encoded.shape[1]), np.arange(X_test_encoded.shape[1]))

# Handle fitur yang hilang pada X_train_encoded
X_train_encoded = np.concatenate((X_train_encoded, np.zeros((X_train_encoded.shape[0], len(missing_features_train)))), axis=1)

# Handle fitur yang hilang pada X_test_encoded
X_test_encoded = np.concatenate((X_test_encoded, np.zeros((X_test_encoded.shape[0], len(missing_features_test)))), axis=1)

In [97]:
# Define a function for cross-validation
def cross_validate_model(model, param_grid, X_train, y_train):
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    return grid_search






# XGBoost
xgboost_param_grid = {'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 5, 7]}
xgboost_grid_search = cross_validate_model(XGBClassifier(random_state=42), xgboost_param_grid, X_train_encoded, y_train)

# Display the best hyperparameters for each model




# Evaluate models with best parameters
def evaluate_with_best_params(model, X_test, y_test):
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))






# Evaluate XGBoost
print("\nXGBoost - Test Set:")
evaluate_with_best_params(xgboost_grid_search.best_estimator_, X_test_encoded, y_test)


XGBoost - Test Set:
              precision    recall  f1-score   support

           0       0.38      0.05      0.09       335
           1       0.58      0.94      0.72       465

    accuracy                           0.57       800
   macro avg       0.48      0.50      0.41       800
weighted avg       0.50      0.57      0.46       800



# Testing Using No Label Data

(1) Upload ecommerce_pengujian.csv

(2) Gunakan  model terbaik yang diperoleh pada percobaan sebelumnya dan lakukan prediksi menggunakan model tersebut

(3) Simpan hasil prediksi pada model tersebut kedalam bentuk csv

(4) Upload hasil prediksi kedalam form submission

In [102]:
xgboost_model = XGBClassifier(colsample_bytree=0.9, random_state=42)
xgboost_model.fit(X_train_encoded, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [105]:
# Menggunakan model XGBoost untuk prediksi pada df test
hasil_prediksi_test = xgboost_model.predict(X_test_encoded)  # Ganti X_test_encoded dengan df test Anda

In [107]:
# Create a new DataFrame with the predictions
result_df = pd.DataFrame({'Prediction': hasil_prediksi_test})

In [108]:
# Save the DataFrame to a CSV file
result_df.to_csv('predictions.csv', index=False)